Installing necessary modules.

In [ ]:
!pip install xmltodict #The module that allows reading of the .xml files

Importing the necessary modules.

In [ ]:
from google.colab import drive
import xmltodict
import pandas as pd
import os
import numpy as np

To be able to access to the files that are going to be used in this code, I have uploaded them to my Google drive. And, by using the code below, I can let Colab access to the documents in my drive.

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Reading the file that contains the synsets of the WordNet items.

In [ ]:
syn_data = open("/content/drive/My Drive/a-synsets-30.xml", "r", encoding="utf-8")
syn_dic = xmltodict.parse(syn_data.read())["syn-list"]
syn_data.close()
print(syn_dic)

{'noun-syn-list': {'noun-syn': [{'@id': 'n#07516905', '@categ': 'wrath'}, {'@id': 'n#07501420', '@categ': 'worship'}, {'@id': 'n#07533735', '@categ': 'world-weariness'}, {'@id': 'n#07509996', '@categ': 'wonder'}, {'@id': 'n#07533877', '@categ': 'woe'}, {'@id': 'n#07506031', '@categ': 'withdrawal'}, {'@id': 'n#07538812', '@categ': 'weight'}, {'@id': 'n#07534278', '@categ': 'weepiness'}, {'@id': 'n#07498614', '@categ': 'weakness'}, {'@id': 'n#07548860', '@categ': 'warpath'}, {'@id': 'n#07546279', '@categ': 'warmheartedness'}, {'@id': 'n#07550891', '@categ': 'vindictiveness'}, {'@id': 'n#07513711', '@categ': 'unrest'}, {'@id': 'n#07502241', '@categ': 'unfriendliness'}, {'@id': 'n#07505538', '@categ': 'neutral-unconcern'}, {'@id': 'n#07523649', '@categ': 'unassertiveness'}, {'@id': 'n#07517550', '@categ': 'umbrage'}, {'@id': 'n#07514520', '@categ': 'tumult'}, {'@id': 'n#07528097', '@categ': 'triumph'}, {'@id': 'n#07522043', '@categ': 'trepidation'}, {'@id': 'n#07515560', '@categ': 'tranqui

Reading the file that contains the hierarchy information of the synsets

In [ ]:
hierarchy_data = open("/content/drive/My Drive/a-hierarchy.xml", "r", encoding="utf-8")
hierarchy_dic = xmltodict.parse(hierarchy_data.read())["categ-list"]["categ"]
hierarchy_data.close()
print(hierarchy_dic)

[{'@name': 'root'}, {'@name': 'mental-state', '@isa': 'root'}, {'@name': 'physical-state', '@isa': 'root'}, {'@name': 'behaviour', '@isa': 'root'}, {'@name': 'situation', '@isa': 'root'}, {'@name': 'signal', '@isa': 'root'}, {'@name': 'trait', '@isa': 'root'}, {'@name': 'sensation', '@isa': 'root'}, {'@name': 'cognitive-state', '@isa': 'mental-state'}, {'@name': 'affective-state', '@isa': 'mental-state'}, {'@name': 'cognitive-affective-state', '@isa': 'mental-state'}, {'@name': 'mood', '@isa': 'affective-state'}, {'@name': 'emotion', '@isa': 'affective-state'}, {'@name': 'emotion-eliciting-situation', '@isa': 'situation'}, {'@name': 'edonic-signal', '@isa': 'signal'}, {'@name': 'positive-emotion', '@isa': 'emotion'}, {'@name': 'negative-emotion', '@isa': 'emotion'}, {'@name': 'neutral-emotion', '@isa': 'emotion'}, {'@name': 'ambiguous-emotion', '@isa': 'emotion'}, {'@name': 'joy', '@isa': 'positive-emotion'}, {'@name': 'love', '@isa': 'positive-emotion'}, {'@name': 'affection', '@isa':

A function that takes a hierarchy dictionary and a category to be mapped to the higher category and goes through the all categories in the dictionary to find the higher category and returns the higher category name. In case that it does not find any higher category, it just returns the initial category.

In [ ]:
def cat_map(hier_d, categ):
  i = 0
  while hier_d[i]['@name'] != categ and i < (len(hier_d)-1):
    i += 1
  if i == len(hier_d):
    return categ
  else:
    if hier_d[i]['@isa'] not in ['positive-emotion', 'negative-emotion', 'neutral-emotion', 'ambiguous-emotion']:
      return hier_d[i]['@isa']
    else:
      return categ

A function that takes a dictionary of the categories and the items, and maps them to the higher category unless their category is already positive, negative, neutral and ambiguous emotion. It returns the manipulated dictionary.

In [ ]:
def hier_map_noun(s_d):
  change = False
  for n in s_d:
    if n['@categ'] not in ['positive-emotion', 'negative-emotion', 'neutral-emotion', 'ambiguous-emotion']:
      if n['@categ'] != cat_map(hierarchy_dic[1:], n['@categ']):
        n['@categ'] = cat_map(hierarchy_dic[1:], n['@categ'])
        change = True
  return s_d, change

A code that maps the categories to the higher one until all of them are mapped and no changes can be done.

In [ ]:
c = True
i = 0
while c:
  i += 1
  m_dic = hier_map_noun(syn_dic['noun-syn-list']['noun-syn'])
  syn_dic['noun-syn-list']['noun-syn'] = m_dic[0]
  c = m_dic[1]
  print(i)
print(syn_dic['noun-syn-list']['noun-syn'])

1
2
3
4
5
6
[{'@id': 'n#07516905', '@categ': 'general-dislike'}, {'@id': 'n#07501420', '@categ': 'love'}, {'@id': 'n#07533735', '@categ': 'sadness'}, {'@id': 'n#07509996', '@categ': 'surprise'}, {'@id': 'n#07533877', '@categ': 'sadness'}, {'@id': 'n#07506031', '@categ': 'neutral-unconcern'}, {'@id': 'n#07538812', '@categ': 'sadness'}, {'@id': 'n#07534278', '@categ': 'sadness'}, {'@id': 'n#07498614', '@categ': 'liking'}, {'@id': 'n#07548860', '@categ': 'general-dislike'}, {'@id': 'n#07546279', '@categ': 'love'}, {'@id': 'n#07550891', '@categ': 'general-dislike'}, {'@id': 'n#07513711', '@categ': 'ambiguous-agitation'}, {'@id': 'n#07502241', '@categ': 'general-dislike'}, {'@id': 'n#07505538', '@categ': 'neutral-unconcern'}, {'@id': 'n#07523649', '@categ': 'negative-fear'}, {'@id': 'n#07517550', '@categ': 'general-dislike'}, {'@id': 'n#07514520', '@categ': 'anxiety'}, {'@id': 'n#07528097', '@categ': 'joy'}, {'@id': 'n#07522043', '@categ': 'negative-fear'}, {'@id': 'n#07515560', '@categ': '

Reading the datasets from all the languages and merging them in the same DataFrame based on their id numbers to form an aligned dataset across languages. It also puts the words with the same id number in the same row for each language.

In [ ]:
directory = '/content/drive/MyDrive/WordNet_data'
multilingual = pd.DataFrame()
for filename in sorted(os.listdir(directory)):
  if 'tab' == filename[-3:]:
    print(filename)
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as f:
      next(f)
      c_n = 'word_'+(filename[8:-4])
      names = ['id','lemma', c_n]
      data = pd.read_csv(f, sep='\t', names=names, on_bad_lines='skip')
      if data[data.lemma == (filename[8:-4]) + ':lemma'].empty:
        data = data[data.lemma == 'lemma']
      else:
        data = data[data.lemma == (filename[8:-4]) + ':lemma']
      data = data.iloc[:,[0,2]]
      d = data.fillna('')
      df = d[['id', c_n]].groupby('id').agg(lambda x: ','.join(x.unique()))
      if multilingual.empty:
        multilingual = df
      else:
        multilingual = multilingual.merge(df, on='id', how='left')
multilingual = multilingual.dropna()

wn-data-eng.tab
wn-data-fin.tab
wn-data-jpn.tab
wn-data-slv.tab
wn-data-spa.tab
wn-data-tha.tab
wn-data-zsm.tab


In [ ]:
print(multilingual)

                                                word_eng  \
id                                                         
00001740-a                                          able   
00001740-n                                        entity   
00002137-n                   abstraction,abstract entity   
00002621-r            barely,hardly,just,scarcely,scarce   
00002684-n                        object,physical object   
...                                                  ...   
13376012-n  credit card,charge card,charge plate,plastic   
13381145-n                          deposit,bank deposit   
13381734-n                       check,bank check,cheque   
13384164-n                                       pension   
13384557-n                                         money   

                                                     word_fin  \
id                                                              
00001740-a                           kykenevä,pystyvä,taitava   
00001740-n              

In [ ]:
print(multilingual.sample())

                                                word_eng  \
id                                                         
10841405-n  Beethoven,van Beethoven,Ludwig van Beethoven   

                                                word_fin            word_jpn  \
id                                                                             
10841405-n  Beethoven,van Beethoven,Ludwig van Beethoven  ルートヴィヒ・ヴァン・ベートーヴェン   

                        word_slv                        word_spa  \
id                                                                 
10841405-n  Ludwig van Beethoven  Beethoven,Ludwig van Beethoven   

                                          word_tha   word_zsm  
id                                                             
10841405-n  บีโธเฟน,ฟาน เบโธเฟน,ลุดวิก ฟาน เบโธเฟน  Beethoven  


A function that gets dictionaries of different word types (i.e., adjectives, nouns, etc.) and map them to their emotional categories according to their ids. For the word types other than nouns, this procedure is completed according to their noun ids.

In [ ]:
def cat_assign(d, noun=True):
  c_d = []
  for c in d:
    l = []
    id = c['@id'][2:] + '-' + c['@id'][0]
    l.append(id)
    if not noun:
      n_id = c['@noun-id'][2:] + '-n'
      categ = np.nan
      for n in n_l:
        if n_id == n[0]:
          categ = n[1]
      l.append(categ)
    else:
      l.append(c['@categ'])
    c_d.append(l)
  return c_d

In [ ]:
print(syn_dic['adj-syn-list']['adj-syn'])

[{'@id': 'a#01737241', '@noun-id': 'n#07514071', '@caus-stat': 'stat'}, {'@id': 'a#00811536', '@noun-id': 'n#07514071', '@caus-stat': 'stat'}, {'@id': 'a#01626562', '@noun-id': 'n#07503987', '@caus-stat': 'caus'}, {'@id': 'a#00480965', '@noun-id': 'n#07507329', '@caus-stat': 'stat'}, {'@id': 'a#01632314', '@noun-id': 'n#07509474', '@caus-stat': 'stat'}, {'@id': 'a#00154163', '@noun-id': 'n#07506569', '@caus-stat': 'stat'}, {'@id': 'a#02405805', '@noun-id': 'n#07522632', '@caus-stat': 'caus'}, {'@id': 'a#02405805', '@noun-id': 'n#07511238', '@caus-stat': 'caus'}, {'@id': 'a#01807605', '@noun-id': 'n#07500741', '@caus-stat': 'caus'}, {'@id': 'a#00904290', '@noun-id': 'n#07500741', '@caus-stat': 'caus'}, {'@id': 'a#00854255', '@noun-id': 'n#07544647', '@caus-stat': 'caus'}, {'@id': 'a#00114266', '@noun-id': 'n#07518878', '@caus-stat': 'stat'}, {'@id': 'a#00085264', '@noun-id': 'n#07513508', '@caus-stat': 'stat'}, {'@id': 'a#00078851', '@noun-id': 'n#07519773', '@caus-stat': 'stat'}, {'@id

Assigning emotional category to the items in the dataset.

In [ ]:
n_l = cat_assign(syn_dic['noun-syn-list']['noun-syn'])
a_l = cat_assign(syn_dic['adj-syn-list']['adj-syn'], noun=False)
v_l = cat_assign(syn_dic['verb-syn-list']['verb-syn'], noun=False)
r_l = cat_assign(syn_dic['adv-syn-list']['adv-syn'], noun=False)
c_l = n_l + a_l + v_l + r_l
c_df = pd.DataFrame(c_l, columns=['id', 'categ'])
c_df = c_df.drop_duplicates()
print(c_df.sample())

             id    categ
654  00704249-v  anxiety


Tagging the emotional categories to the items in the dataset.

In [ ]:
multilingual = multilingual.merge(c_df, on='id', how='left')

In [ ]:
print(multilingual.sample())

              id                  word_eng                  word_fin  \
6266  09092497-n  Maine,Pine Tree State,ME  Maine,Pine Tree State,ME   

               word_jpn word_slv  word_spa         word_tha  \
6266  メーン,マツの州,メーン州,松の州    Maine  Maine,me  มลรัฐเมน,รัฐเมน   

                            word_zsm categ  
6266  Maine,Pine Tree State,aku,saya   NaN  


Filtering out the words in the database that has an emotional category.

In [ ]:
emo_data = multilingual[multilingual.categ.notnull()]
print(emo_data)

              id                                           word_eng  \
71    00029933-a  avaricious,covetous,grabby,grasping,greedy,pre...   
208   00079485-a                                            fearful   
211   00080357-a  panicky,panicked,panic-stricken,panic-struck,t...   
214   00081671-a                                  unafraid,fearless   
231   00089550-a  annoying,bothersome,galling,irritating,nettles...   
...          ...                                                ...   
5186  07554856-n                                            empathy   
5187  07555014-n                                         enthusiasm   
5188  07555184-n                eagerness,avidity,avidness,keenness   
5189  07555402-n                             ardor,ardour,elan,zeal   
5190  07555647-n                                         exuberance   

                                               word_fin  \
71                    ahne,rahanahne,himokas,himoitseva   
208                          

In [ ]:
print(emo_data.sample())

              id                              word_eng  \
5168  07542675-n  discouragement,disheartenment,dismay   

                      word_fin                      word_jpn   word_slv  \
5168  tyrmistys,masennus,pelko  力落とし,力落し,大腐り,大腐,挫折,挫折感,落胆,銷魂  malodušje   

                 word_spa                             word_tha  \
5168  desaliento,desánimo  ความท้อแท้,ความท้อใจ,ความหมดกำลังใจ   

                                               word_zsm    categ  
5168  kehampaan,kekecewaan,kekesalan,kemurungan,putu...  despair  


Saving the resulting dataset.

In [ ]:
emo_data.to_csv('MultilingualData.csv', encoding='utf-8')

In [ ]:
!cp MultilingualData.csv /content/drive/MyDrive/